In [4]:
pip install faker


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.2 MB/s eta 0:00:00


In [5]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

In [6]:
# Configuración de Faker

fake = Faker('es_ES')

# Configuración del generador de números aleatorios
random.seed(42)

# Crear una lista para almacenar los datos
data = []
# Generar 1000 registros
for _ in range(1000):
    dni = random.randint(10000000, 99999999)  # Generar DNI de 8 dígitos
    nombre = fake.first_name()
    apellido = fake.last_name()
    edad = random.randint(70, 90)
    fecha_nacimiento = fake.date_of_birth(minimum_age=edad)
    id_solicitud = random.randint(1, 1000)
    usuario_solicitante = dni

    data.append([dni, nombre, apellido, edad, fecha_nacimiento, id_solicitud, usuario_solicitante])

# Crear un DataFrame con los datos
df = pd.DataFrame(data, columns=['dni', 'nombre', 'apellido', 'edad', 'fecha_de_nacimiento', 'id_solicitud', 'usuario_solicitante'])


# Guardar el DataFrame en un archivo CSV
csv=df.to_csv('dataset.csv', index=False)

df

,dni,nombre,apellido,edad,fecha_de_nacimiento,id_solicitud,usuario_solicitante
0,95822412,Marianela,Echeverría,73,1939-03-04,26,95822412
1,46913810,Cirino,Perales,77,1927-04-29,229,46913810
2,28728463,Damián,Moliner,73,1912-08-01,693,28728463
3,83197857,Zacarías,Robles,72,1924-07-17,605,83197857
4,66629388,José,Marcos,71,1940-05-19,31,66629388
...,...,...,...,...,...,...,...
995,82199509,Salomón,Esteve,85,1931-07-10,247,82199509
996,40245233,Aarón,Miralles,79,1943-01-04,879,40245233
997,29374563,Eligia,Seguí,84,1916-10-27,937,29374563
998,17734153,Camila,Pelayo,87,1920-05-07,423,17734153


In [ ]:
import psycopg2
from psycopg2 import sql  # Añadir esta línea
import pandas as pd

dbname = "obqrzyqi"
user = "obqrzyqi"
password = "8PKTas-pu_I7k1Gjm0ofLxcejh6WqloY"
host = "berry.db.elephantsql.com"
port = "5432"

# Crear una conexión
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

try:

    # Crear una consulta de inserción
    insert_query = sql.SQL("INSERT INTO usuarios ({}) VALUES ({})").format(
        sql.SQL(', ').join(map(sql.Identifier, df.columns)),
        sql.SQL(', ').join(sql.Placeholder() * len(df.columns))
    )

    # Obtener el cursor
    cursor = conn.cursor()

    # Insertar filas del DataFrame en la tabla de la base de datos
    for _, row in df.iterrows():
        cursor.execute(insert_query, tuple(row))

    # Confirmar la transacción
    conn.commit()

    print("Datos insertados correctamente en la tabla.")

except Exception as e:
    print(f"Error: {e}")

finally:
    # Cerrar la conexión
    if conn is not None:
        conn.close()